In [1]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import operator
from mpl_toolkits import mplot3d
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Adjust size of plots
plt.rcParams['figure.figsize'] = [6, 4]

# Multivariate Regression

## Notation

We have a given dataset 

$$
    (x,y) \in \mathbb{R}^{m \times (n+1)},
$$

where $m$ is the number of samples and $n$ is the number of features.

In the dateset we have the features

$$
    x \in \mathbb{R}^{m \times n}
$$

and the targets 

$$
    y \in \mathbb{R}^m.
$$

We say that 

$$
    (x^{(i)}, y^{(i)}) \in \mathbb{R}^{1 \times (n+1)}
$$

is the $i$-th example ($i = 1, \dots, m$) and 

$$
    x_j \in \mathbb{R}^m
$$ 

is the $j$-th feature vector ($j = 1, \dots, n$), such that 

$$
    x^{(i)}_j \in \mathbb{R}  
$$

is the $j$-th feature of the $i$-th example.

## Boston dataset

In [3]:
# Load dataset
data = load_boston()

# Create dataframe
df = pd.DataFrame(data.data, columns=data.feature_names)
df['Target'] = data.target
df.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Target
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,11.9


In [4]:
# Define feature index
feat_idx = [12, 10]

# Split data table into data X and class labels y
X = df.iloc[:, feat_idx].values
y = df.iloc[:, -1].values

# Set names of labels for plot
xlabel = df.columns[feat_idx[0]]
ylabel = df.columns[feat_idx[1]]
zlabel = df.columns[-1]

# Plot data
fig = plt.figure(figsize=(6, 4))
ax = plt.axes(projection ='3d', xlabel=xlabel, ylabel=ylabel, zlabel=zlabel)
ax.scatter(X[:, 0], X[:, 1], y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# Split data in training and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, shuffle=False)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((354, 2), (152, 2), (354,), (152,))

## Custom Implementation

### Hypothesis

The hypothesis is given by

$$
    h_{w, b}(x) = \sum_{j=1}^n w_j x_j + b = w_1 x_1 + w_2 x_2 + \dots + w_n x_n + b, 
$$

where $w_1, w_2, \dots, w_n \in \mathbb{R}^n$, with $n$ the number of features, are the weights and $b \in \mathbb{R}$ is the bias.

We can also write

$$
    h_{w}(x) = \sum_{j=0}^n w_j x_j = w_0 + w_1 x_1 + w_2 x_2 + \dots + w_n x_n, 
$$

if we set $w_0 := b$ and $x_0 := 1$.


In [6]:
# Compute the hypothesis and return as shape (n_samples,)
hypo = lambda X, w, b: np.sum(w * X, axis=1) + b

### Cost function

The Mean Squared Error is given by

$$
\begin{align*}
    MSE(w,b) &= \frac{1}{m} \sum\limits_{i=1}^m (y^{(i)} - h_{w, b}(x^{(i)}))^2\\
             &= \frac{1}{m} \sum\limits_{i=1}^m (y^{(i)} - \sum_{j=1}^n w_j x_j^{(i)} + b)^2,
\end{align*}
$$

where $m$ is the number of samples and $n$ is the number of features.

In [7]:
# Compute the cost function (mean squared error)
mse = lambda X, y, w, b: np.mean(np.square(y - hypo(X, w, b)))

### Gradient Descent

#### Goal: 

$$
    \underset{w \in \mathbb{R}, ~ b \in \mathbb{R}}{\textbf{minimize}} MSE(w,b)
$$

#### Idea: 

$$
\begin{align*}
    w &= w - \alpha \cdot \frac{\partial}{\partial w} MSE(w,b) \\
    b &= b - \alpha \cdot \frac{\partial}{\partial b} MSE(w,b)
\end{align*}
$$

#### Partial derivaties:

$$
\begin{align*}
    \frac{\partial}{\partial w} MSE(w,b) &= \frac{2}{m} \sum\limits_{i=1}^m x^{(i)} (y^{(i)} - h_{w,b}(x^{(i)})) \\
    \frac{\partial}{\partial b} MSE(w,b) &= \frac{2}{m} \sum\limits_{i=1}^m y^{(i)} - h_{w,b}(x^{(i)})
\end{align*}
$$

In [8]:
def gradient_descent(X, y, w, b, alpha, num_iters):  
    """Simple gradient descent.
    
    TODO: Plot.
    
    Parameter
    ---------
        X : ndarray of shpae (n_samples, n_features)
            Input data.
        y : ndarray of shape (n_samples,)
            Labels.
        w : float
            Weight.
        b : float
            Bias.
    
    Return
    ------
        w : float
            Updated weight.
        b : float
            Updated bias.
    """
    # Number of samples
    m = len(X)
    
    # Iteratively update the weight and bias
    for i in range(0, num_iters):
        
        # Compute predictions (for all samples)
        predictions = hypo(X, w, b)
        
        # Compute residuals (for all samples)
        residuals = predictions - y
        
        # Compute partial derivitaves 
        w_gradient = np.mean(residuals.reshape(-1, 1) * X, axis=0)
        b_gradient = np.mean(residuals)

        # Update weigths and bias
        w = w - alpha * w_gradient
        b = b - alpha * b_gradient
        
    return w, b


In [9]:
# Initialize weights and bias
w = np.zeros(X_train.shape[1])
b = 0.0

# Set number of iterations and stepsize alpha
num_iters, alpha = 100000, 0.001

# Run gradient descent
w, b = gradient_descent(X_train, y_train, w, b, alpha,
                        num_iters) 
w, b

(array([-0.86588896, -0.5179788 ]), 43.02261477799343)

In [10]:
def plot(X, y, w, b):
    
    # Plot data
    fig = plt.figure(figsize=(6, 4))
    ax = plt.axes(projection ='3d', xlabel=xlabel, ylabel=ylabel, zlabel=zlabel)
    ax.scatter(X[:, 0], X[:, 1], y)

    # # Add predictions
    # ax.scatter(X_train[:, 0], X_train[:, 1], , color='r')

    # Add regression plane by creating a meshgrid and make predictions
    # on every meshgrid point
    stepsize = 2
    x_min, x_max = min(X[:, 0]), max(X[:, 0])
    y_min, y_max = min(X[:, 1]), max(X[:, 1])
    xx = np.linspace(x_min, x_max, stepsize)
    yy = np.linspace(y_min, y_max, stepsize)
    xx, yy = np.meshgrid(xx, yy)
    zz = hypo(np.stack((xx.ravel(), yy.ravel()), axis=1), w, b)
    ax.plot_surface(xx, yy, zz.reshape(xx.shape), color='r', alpha=.3)

In [11]:
# Plot training data and regression plane
plot(X_train, y_train, w, b)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# Compute training MSE
mse(X_train, y_train, w, b)

34.69862921547744

In [13]:
# Plot test data and regression plane
plot(X_test, y_test, w, b)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Compute test MSE
mse(X_test, y_test, w, b)

35.11147826004231

### $R^2$ score

The residual sum of squares is defined by

$$
    SS_{res} = \sum_{i=1}^m (y^{(i)} - h_{w,b}(x^{(i)}))^2.
$$

The total sum of squares is defined by

$$
    SS_{res} = \sum_{i=1}^m (y^{(i)} - \frac{1}{n} \sum_{i=1}^n y^{(i)})^2.
$$

The $R^2$ score (or coefficient of determenation) is then defined by

$$
    R^2 = 1 - {SS_{\rm res}\over SS_{\rm tot}}.
$$

In [15]:
# Function for residual sum of squares
ss_res = lambda X, y, w, b: np.sum(np.square(y - hypo(X, w, b)))

# Function for total sum of squares
ss_tot = lambda y: np.sum(np.square(y - np.mean(y)))

# Function for coefficient of determination aka R^2
r2 = lambda X, y, w, b: 1 - ss_res(X, y, w, b) / ss_tot(y)

In [16]:
# Compute training R2
r2(X_train, y_train, w, b)

0.5111988633439439

In [17]:
# Compute test R2
r2(X_test, y_test, w, b)

0.4707844625878962

## Scikit-learn

In [18]:
# Define linear regression model (wraps least-squares from SciPy)
reg = LinearRegression()

# Train model
reg.fit(X_train, y_train)

# Show weight and bias
reg.coef_, reg.intercept_

(array([-0.86689097, -1.20571028]), 55.39731766394296)

In [19]:
# Make training predictions
y_pred = reg.predict(X_train)

# Compute training MSE and R2
mean_squared_error(y_train, y_pred), r2_score(y_train, y_pred)

(32.39915613408146, 0.5435916431542485)

In [20]:
# Make test predictions
y_pred = reg.predict(X_test)

# Compute test MSE and R2
mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)

(36.04305587072861, 0.45674331791636913)